In [2]:
import re
f = open("alldeq14.txt", "r+")
deqsstr = f.read()
deqslist = re.split("\n", deqsstr)

In [3]:
#deqslist= re.findall("deq14.b.*", deqsstr)
#deqslist = '\n'.join(deqssample)
#print(deqssample)
#len(deqssample)

In [4]:
from urllib import request

emailstext = ''

s3_root = "https://flint-text.s3.amazonaws.com/"
for deq in deqslist:       
    url = s3_root + deq
    response = request.urlopen(url)
    raw_text = response.read().decode('utf8')
    emailstext = "".join((raw_text, emailstext)) #changed this line

In [5]:
#print(emailstext) #see full email output

In [74]:
froms = re.findall("From:.*[A-Za-z].*", emailstext)
fromstr = '\n'.join(froms)
def refroms(fromstr):
    fromstr_orgs = re.sub("From:\W","", fromstr) #with (ORG)
    return fromstr_orgs

newfromstr = str(refroms(fromstr))
#print(newfromstr)

In [7]:
ons = re.findall("On.*wro[tf]e", emailstext)
onstr = '\n'.join(ons)
def reons(onstr):
    onstr1 = re.sub("On\s.*at\s[0-9]*:[0-9][0-9]\s[AP]M\W*","", onstr)
    onstr2 = re.sub("On\s.*\s[0-9]*","", onstr1)
    onstr3 = re.sub("[0-9]","",onstr2)
    onstr4 = re.sub("wrote","",onstr3)
    return onstr4
    
newonstr = str(reons(onstr))
#print(newonstr)

In [73]:
roughlist = []
fromsenders = re.split('\n', newfromstr) #splits from str into list
onsenders = re.split('\n',newonstr) #splits on... str into list

#combines the two lists; roughlist contains names found in From: header and On... header
roughlist.extend(onsenders)
roughlist.extend(fromsenders)

roughstr = '\n'.join(roughlist) #makes list into string
print("rough list of " + str(len(roughlist)) + " things that might be names:")
print("- - -")
#print(roughstr)

rough list of 2141 things that might be names:
- - -


In [54]:
ultimatelist = [] #creates final list for final sender names to be put into
ultimatelist.clear()

In [72]:
# extracts sender lines formatted in [Last, First] 
lastcommafirstdude = re.findall("[A-Za-z]+,\s[A-Za-z]+.*",roughstr)
ultimatelist.extend(lastcommafirstdude) #puts these names into ultimatelist
#print(lastcommafirstdude)
len(lastcommafirstdude) #number of names already in LAST, FIRST format

1678

In [71]:
# extracts sender lines formatted in [First Last] 
firstlast = re.findall("[A-Z][a-z]+.[A-Z][a-z][A-Za-z]*",roughstr)
#print(firstlast)
len(firstlast)

540

In [57]:
#problem: for "Shekter Smith, Liane", "Shekter Smith" is read as a [First Last]
#band-aid solution:
unwanted = {'Shekter Smith', 'Mary Beth','Tracy Jo'}
firstlast1 = [e for e in firstlast if e not in unwanted]
firstlaststring = '\n'.join(firstlast1)
#print(firstlaststring)

In [58]:
#LIST OF ALL NAMES (FIRST LAST FIRST LAST etc.)
allflnames = re.findall('[A-Z][a-z][A-Za-z]+',firstlaststring)
#print(allflnames)
#len(allflnames)

In [59]:
#LIST OF FIRST NAMES
firstnames = re.findall('^[A-Z][a-z]+\s|\n[A-Z][a-z]+\s',firstlaststring)
firstnames1 = ' '.join(firstnames)
firstnames2 = re.sub("\n","",firstnames1)
firstnameslist = re.split("\s+",firstnames2)
firstnameslist.pop()
#print("first names:")
#print(firstnameslist)
#print("")

#LIST OF LAST NAMES
lastnames1 = re.findall('.[A-Z][a-z]+',firstlaststring)
lastnames2 = ''.join(lastnames1)
lastnameslist = re.split('\s',lastnames2)
del lastnameslist[0]
#print("last names:")
#print(lastnameslist)
#print(" ")
#if len(lastnameslist) == len(firstnameslist):
    #print("yay, there are " + str(len(lastnameslist)) + " first and last names.")
#else:
    #print("drat! the number of first names found doesn't match the number of last names found")

In [70]:
print("Reformatted names:")
print("- - -")
for idx, value in enumerate(firstnameslist):
    key = lastnameslist[idx] + ", " + firstnameslist[idx]
    #print(key)
    ultimatelist.append(key) #adds these reformatted names to ultimatelist

Reformatted names:
- - -


In [61]:
len(ultimatelist)

2091

In [62]:
print("ULTIMATE LIST of SENDER NAMES in [Last, First]:")
print("- - -")
ultimateliststr = '\n'.join(ultimatelist)
print(ultimateliststr)
# final list :o) 

ULTIMATE LIST of SENDER NAMES in [Last, First]:
- - -
Cook, Pat (DEQ) <COOKP michigan.gov> 
Rosenthal, Adam (DEQ) <ROSENTHAILA@michigan,pov> 
Rosenthal, Adam (DEQ) <ROSENTHALA@michigan,gov> 
Smith, Liane (DEQY) <SHEKTERL@. michigan.gov> 
Prysby, Mike (DEQ) <PRYSBYM@michigan.gov> 
Cook, Pat {DEQ) <COOKP@michigan.gov> 
Rosenthal, Adam (DEQ) <ROSENTHALA@michigan.gov> wrofe
Rosenthal, Adam (DEQ) <ROSENTHALA@michigan.gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan.gov> 
Smith, Liane (DEQ) <SHEKTERL @michigan.gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan. gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan.gov> 
Thelen, Mary Beth (DEQ) <THELENMZ@michigan.gov> 
Stanton, Terry A. (Treasury)" <StantonT@michigan.gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan.gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan.gov> 
Wurfel, Brad (DEQ) <WurfelB@michigan gov> 
Rosenthal, Adam (DEQ) <ROSENTHAILA@michigan,pov> 
Rosenthal, Adam (DEQ) <ROSENTHALA@michigan,gov> 
Smith, Liane (DEQY) <SHEKTERL@. michigan.gov> 
Prysby, Mike (DEQ) <PRYS

In [77]:
ultimatelist_senders = list(dict.fromkeys(ultimatelist))

#print(str(len(ultimatelist_senders)) + " unique senders found:")
#print("")
#print(ultimatelist_senders)
ultimatelist_sendersstr = "\n".join(ultimatelist_senders)
#print(ultimatelist_sendersstr)

In [64]:
# # # descriptives # # #

In [65]:
froms.clear()
ons.clear()
froms = re.findall("From:.*[A-Za-z].*", emailstext)
ons = re.findall("On.*wro[tf]e", emailstext)
print((str(len(ons) + len(froms))) + " total froms and ons vs. " + str(len(ultimatelist)) + " senders in ultimatelist")
percent = str(round(((len(ultimatelist))/(len(ons) + len(froms))*100),2))
print(percent + "%")

2154 total froms and ons vs. 2091 senders in ultimatelist
97.08%


In [66]:
fromsandonsnames = (len(onsenders)+len(fromsenders))
percentofons = (((len(onsenders)) / fromsandonsnames) * 100)
percentofonsr = round(percentofons,2)
print(str(percentofonsr) + "% estimated to have an On... header")

9.67% estimated to have an On... header


In [69]:
#Wurfel errors
#ult = "\n".join(ultimatelist_senders)
#re.findall("W.+rfel",ult)